# Chapter 24. MapReduce

In [3]:
from __future__ import division
import math, random, re, datetime
from collections import defaultdict, Counter
from functools import partial
from naive_bayes import tokenize

[MapReduce](https://en.wikipedia.org/wiki/MapReduce) is a programming model for performing parallel processing on large data sets.  
Imagine that we have a (very large) collection that we would like to process in some way.  
For example, the items might be website logs, the texts of various books, image files, or anything else.  
A basic version of the MapReduce algorithm consists of the following steps:
1. Use a `mapper` function to turn each item into zero or more key-value pairs. This is often called the `map()` function, but Python already has a function called `map()`, and we don't want to confuse the two.
2. Collect together all of the pairs with identical keys.
3. Use a `reducer()` function on each collection of grouped values to produce output values for the corresponding key.  

Let's examine this process more concretely using an example that involves counting words.

## Example: Word Count